<a href="https://colab.research.google.com/github/Kimhansav/everynocode_search_engine/blob/main/BP_sentence_embed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#질답세션의 임베딩 벡터를 추가한 뒤 .csv로 저장하는 코드(로컬)

In [ ]:
!pip install faiss-cpu #서버 업로드 시 삭제
!pip install -U sentence-transformers #서버 업로드 시 삭제

import numpy as np
import os
import pandas as pd
import urllib.request
import faiss
import time
import torch
import json
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from transformers import AutoModel, AutoTokenizer
from google.colab import drive
from transformers import RobertaTokenizerFast, RobertaForSequenceClassification, TextClassificationPipeline, pipeline
import torch.nn as nn
!pip install datasets

In [ ]:
from transformers import RobertaTokenizerFast, RobertaForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset, load_dataset
import torch
from google.colab import drive
from sklearn.model_selection import StratifiedKFold, train_test_split

In [ ]:
#구글 드라이브 마운트
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#.csv파일 불러오기
file_path = '/content/drive/My Drive/summary_result.csv'
df = pd.read_csv(file_path)

#데이터프레임의 question 열을 리스트로 변형
data = df.question.to_list()

In [ ]:
#모델 로드
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
HUGGINGFACE_MODEL_PATH = 'BM-K/KoSimCSE-roberta-multitask'

model = AutoModel.from_pretrained(HUGGINGFACE_MODEL_PATH)
tokenizer = AutoTokenizer.from_pretrained(HUGGINGFACE_MODEL_PATH)
model.to(device)

In [ ]:
#수동 배치처리
batch_size = 64

result = []
for i in tqdm(range(0, len(data), batch_size), desc = "Embedding"):
    batch_texts = data[i : i + batch_size]
    inputs = tokenizer(batch_texts, padding = True, truncation = True, return_tensors = 'pt')
    inputs = {k : v.to(device) for k, v in inputs.items()}
    #모델 실행
    with torch.no_grad():
        outputs = model(**inputs)

    mean_pooling = torch.mean(outputs.last_hidden_state, dim = 1)
    result.append(mean_pooling)

print(result)

In [ ]:
print(len(result))
print(len(result[0]))
print(len(result[0][0]))

In [ ]:
#데이터프레임에 새로운 열로 추가
df['embedding'] = [sentence.cpu().numpy() for batches in result for sentence in batches]

#csv파일로 저장할 때 쉼표가 사라지지 않게 해야 함, json.dumps 사용
df['embedding'] = df['embedding'].apply(lambda x: json.dumps(x.tolist()))

In [ ]:
print(len(df['embedding'][55]))
print(df['embedding'])

In [ ]:
#.csv 파일로 google drive에 저장 (나중에 google cloud storage에 저장해야 함)
save_path = '/content/drive/My Drive/sentence_embed_result.csv'

df.to_csv(save_path)

In [ ]:
import ast
#로컬용 FAISS 코드
#데이터 불러오기
test_file_path = '/content/drive/My Drive/sentence_embed_result.csv'
df = pd.read_csv(test_file_path)

#csv파일로 저장할 때 쉼표를 살리기 위해 json.dumps로 저장했기 때문에 json.loads 사용
df['embedding'] = df['embedding'].apply(json.loads)
encoded_data = torch.tensor(df['embedding'].tolist())
data = df.drop(['embedding'], axis = 1)

#FAISS에 등록
index = faiss.IndexIDMap(faiss.IndexFlatIP(768))
print(encoded_data)
index.add_with_ids(encoded_data, np.array(range(0, len(data))))
faiss.write_index(index, 'question-answer')


In [ ]:
#검색 함수에서 사용할 사용자 입력 인코딩 함수
def encode(query):
  inputs = tokenizer(query, padding = True, truncation = True, return_tensors = 'pt')

  with torch.no_grad():
        outputs = model(**inputs)

  #배치 단위가 아니라 단일 문장이기 때문에 mean pooling을 할 때 dim = 0으로 설정
  mean_pooling = torch.mean(outputs.last_hidden_state, dim = 0)
  return mean_pooling

In [ ]:
def search(query):
  t = time.time()
  query_vector = encode(query)
  k = 20
  top_k = index.search(query_vector, k)
  print('total time: {}'.format(time.time() - t))

  return [{'question': data.iloc[_id]['question'], 'answer': data.iloc[_id]['answer'], 'question_summary': data.iloc[_id]['question_summary'], 'answer_summary': data.iloc[_id]['answer_summary']} for _id in top_k[1].tolist()[0]]

In [ ]:
query = str(input())
results = search(query)
print('results :')

for result in results:
  print('\t Question summary :', result['question_summary'], '\n')
  print('\t Answer summary :', result['answer_summary'], '\n')
  print('\t Question :', result['question'], '\n')
  print('\t Answer :', result['answer'], '\n\n\n')